<a href="https://colab.research.google.com/github/Joseph-dh-woods/IGDB-CSV-generator/blob/main/IGDB_API_user_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run this cell first to install the IGDB API

In [1]:
pip install igdb-api-v4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Run this cell next to set up some required stuff

In [2]:
#@title Imports and Function Definitions
import requests
from igdb.wrapper import IGDBWrapper
import json
import time
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import re
import datetime
from IPython.display import clear_output 
import random
def makeBar(perc):
  barStart = '|\x1b[42m'
  barEnd = '|'
  loadEnd = '\x1b[0m'
  barFill = ''
  barMid = ''
  for _ in range(perc):
    barMid += ' '
  barWhole = barStart+barMid+loadEnd+barEnd
  while (len(barWhole) < 32):
    barFill += ' '
    barWhole = barStart+barMid+loadEnd+barFill+barEnd
  barWhole = barWhole + '\n' +barWhole
  return barWhole
def progressBar(num, den):
  perc = num/den
  perc = perc*100
  perc = round(perc/5)
  print(makeBar(perc))
  r = random.randint(6,10)
  tm = (den-num)*r
  print(str(tm//60) + ' Minutes and ' + str(tm%60) + ' Seconds left')
def unicodeReplace(found):
  found = found.replace('\\u0026', '&')
  found = found.replace('\\u0027', "'")
  found = found.replace('\\u2022', "•")
  found = found.replace('•', '-')
  found = found.replace('\"', '"')
  found = found.replace('\n', ' ')
  return found
def getID(inGame):
  found = re.search('"id": (.+?),', inGame).group(1)
  return found
def getName(inGame):
  found = re.search('"name": "(.+?)"', inGame).group(1)
  found = unicodeReplace(found)
  return found
def getCollection(inGame):
  found = re.search('"collection": (.+?),', inGame).group(1)
  found = found.replace(',', '')
  idList = found.split()
  found = ''
  for numin in idList:
    byteArray = wrapper.api_request('collections', 'fields *; where id = ' + str(numin) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    myJson = myJson.replace('\n', ' ')
    myJson = ' '.join(myJson.split())
    plainTxt = re.search('"name": "(.+?)"', myJson).group(1)
    found = found + plainTxt + ', '
  found = found.rstrip()
  found = found.strip(',')
  found = unicodeReplace(found)
  return found
def getRelease(inGame):
  found = re.search('"first_release_date": (.+?),', inGame).group(1)
  timestamp = datetime.datetime.fromtimestamp(int(found))
  return timestamp.strftime('%Y-%m-%d')
def getGenres(inGame):
  found = re.search('"genres": \[(.+?)\]', inGame).group(1)
  found = found.replace(',', '')
  idList = found.split()
  found = ''
  for numin in idList:
    byteArray = wrapper.api_request('genres', 'fields *; where id = ' + str(numin) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    myJson = myJson.replace('\n', ' ')
    myJson = ' '.join(myJson.split())
    plainTxt = re.search('"name": "(.+?)"', myJson).group(1)
    found = found + plainTxt + ', '
    time.sleep(.1)
  found = found.rstrip()
  found = found.strip(',')
  found = unicodeReplace(found)
  return found
def getThemes(inGame):
  found = re.search('"themes": \[(.+?)\]', inGame).group(1)
  found = found.replace(',', '')
  idList = found.split()
  found = ''
  for numin in idList:
    byteArray = wrapper.api_request('themes', 'fields *; where id = ' + str(numin) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    myJson = myJson.replace('\n', ' ')
    myJson = ' '.join(myJson.split())
    plainTxt = re.search('"name": "(.+?)"', myJson).group(1)
    found = found + plainTxt + ', '
  found = found.rstrip()
  found = found.strip(',')
  found = unicodeReplace(found)
  return found
def getCompanies(inGame):
  found = re.search('"involved_companies": \[(.+?)\]', inGame).group(1)
  found = found.replace(',', '')
  idList = found.split()
  found = ''
  for numin in idList:
    byteArray = wrapper.api_request('involved_companies', 'fields *; where id = ' + str(numin) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    myJson = myJson.replace('\n', ' ')
    myJson = ' '.join(myJson.split())
    plainTxt = re.search('"company": (.+?),', myJson).group(1)
    byteArray2 = wrapper.api_request('companies', 'fields *; where id = ' + str(plainTxt) + ';')
    nameByte = byteArray2.decode('utf8').replace("'", '"')
    plainName = re.search('"name": "(.+?)"', nameByte).group(1)
    found = found + plainName
    if '"developer": true,' in myJson:
      found += "(DEV)"
    if '"publisher": true,' in myJson:
      found += "(PUB)"
    if '"porting": true,' in myJson:
      found += "(PORT)"
    if '"supporting": true,' in myJson:
      found += "(SPRT)" 
    found += ', '
  found = found.rstrip()
  found = found.strip(',')
  found = unicodeReplace(found)
  return found
def getSummary(inGame):
  found = re.search('summary": "(.+?)",', inGame).group(1)
  found = unicodeReplace(found)
  return found
def getFranchises(inGame):
  found = re.search('franchises": \[(.+?)\]', inGame).group(1)
  found = found.replace(',', '')
  idList = found.split()
  found = ''
  for numin in idList:
    byteArray = wrapper.api_request('franchises', 'fields *; where id = ' + str(numin) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    myJson = myJson.replace('\n', ' ')
    myJson = ' '.join(myJson.split())
    plainTxt = re.search('"name": "(.+?)"', myJson).group(1)
    found = found + plainTxt + ', '
  found = found.rstrip()
  found = found.strip(',')
  found = unicodeReplace(found)
  return found
def getCover(inGame):
  byteArray = wrapper.api_request('games', 'fields cover; where id = ' + str(inGame) + ';')
  myJson = byteArray.decode('utf8').replace("'", '"')
  if "cover" in myJson:
    plaintext = re.search('"cover": (.+?)\n', myJson).group(1)
    byteArray = wrapper.api_request('covers', 'fields *; where id = ' + str(plaintext) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    plaintext = re.search('"image_id": "(.+?)"', myJson).group(1)
    found = "https://images.igdb.com/igdb/image/upload/t_cover_big/" + plaintext + ".png"
    return found
  else:
    return "no Cover found"
def getPlatforms(gameID):
  consoleID = { "3":"Linux",
"4":"Nintendo 64",
"5":"Wii",
"6":"PC (Microsoft Windows)",
"7":"PlayStation",
"8":"PlayStation 2",
"9":"PlayStation 3",
"11":"Xbox",
"12":"Xbox 360",
"13":"PC DOS",
"14":"Mac",
"15":"Commodore C64/128",
"16":"Amiga",
"18":"Nintendo Entertainment System (NES)",
"19":"Super Nintendo Entertainment System (SNES)",
"20":"Nintendo DS",
"21":"Nintendo GameCube",
"22":"Game Boy Color",
"23":"Dreamcast",
"24":"Game Boy Advance",
"25":"Amstrad CPC",
"26":"ZX Spectrum",
"27":"MSX",
"29":"Sega Mega Drive/Genesis",
"30":"Sega 32X",
"32":"Sega Saturn",
"33":"Game Boy",
"34":"Android",
"35":"Sega Game Gear",
"37":"Nintendo 3DS",
"38":"PlayStation Portable",
"39":"iOS",
"41":"Wii U",
"42":"N-Gage",
"44":"Tapwave Zodiac",
"46":"PlayStation Vita",
"47":"Virtual Console (Nintendo)",
"48":"PlayStation 4",
"49":"Xbox One",
"50":"3DO Interactive Multiplayer",
"51":"Family Computer Disk System",
"52":"Arcade",
"53":"MSX2",
"55":"Legacy Mobile Device",
"57":"WonderSwan",
"58":"Super Famicom",
"59":"Atari 2600",
"60":"Atari 7800",
"61":"Atari Lynx",
"62":"Atari Jaguar",
"63":"Atari ST/STE",
"64":"Sega Master System",
"65":"Atari 8-bit",
"66":"Atari 5200",
"67":"Intellivision",
"68":"ColecoVision",
"69":"BBC Microcomputer System",
"70":"Vectrex",
"71":"Commodore VIC-20",
"72":"Ouya",
"73":"BlackBerry OS",
"74":"Windows Phone",
"75":"Apple II",
"77":"Sharp X1",
"78":"Sega CD",
"79":"Neo Geo MVS",
"80":"Neo Geo AES",
"82":"Web browser",
"84":"SG-1000",
"85":"Donner Model 30",
"86":"TurboGrafx-16/PC Engine",
"87":"Virtual Boy",
"88":"Odyssey",
"89":"Microvision",
"90":"Commodore PET",
"91":"Bally Astrocade",
"93":"Commodore 16",
"94":"Commodore Plus/4",
"95":"PDP-1",
"96":"PDP-10",
"97":"PDP-8",
"98":"DEC GT40",
"99":"Family Computer",
"100":"Analogue electronics",
"101":"Ferranti Nimrod Computer",
"102":"EDSAC",
"103":"PDP-7",
"104":"HP 2100",
"105":"HP 3000",
"106":"SDS Sigma 7",
"107":"Call-A-Computer time-shared mainframe computer system",
"108":"PDP-11",
"109":"CDC Cyber 70",
"110":"PLATO",
"111":"Imlac PDS-1",
"112":"Microcomputer",
"113":"OnLive Game System",
"114":"Amiga CD32",
"115":"Apple IIGS",
"116":"Acorn Archimedes",
"117":"Philips CD-i",
"118":"FM Towns",
"119":"Neo Geo Pocket",
"120":"Neo Geo Pocket Color",
"121":"Sharp X68000",
"122":"Nuon",
"123":"WonderSwan Color",
"124":"SwanCrystal",
"125":"PC-8801",
"126":"TRS-80",
"127":"Fairchild Channel F",
"128":"PC Engine SuperGrafx",
"129":"Texas Instruments TI-99",
"130":"Nintendo Switch",
"131":"Nintendo PlayStation",
"132":"Amazon Fire TV",
"133":"Philips Videopac G7000",
"134":"Acorn Electron",
"135":"Hyper Neo Geo 64",
"136":"Neo Geo CD",
"137":"New Nintendo 3DS",
"138":"VC 4000",
"139":"1292 Advanced Programmable Video System",
"140":"AY-3-8500",
"141":"AY-3-8610",
"142":"PC-50X Family",
"143":"AY-3-8760",
"144":"AY-3-8710",
"145":"AY-3-8603",
"146":"AY-3-8605",
"147":"AY-3-8606",
"148":"AY-3-8607",
"149":"PC-98",
"150":"Turbografx-16/PC Engine CD",
"151":"TRS-80 Color Computer",
"152":"FM-7",
"153":"Dragon 32/64",
"154":"Amstrad PCW",
"155":"Tatung Einstein",
"156":"Thomson MO5",
"157":"NEC PC-6000 Series",
"158":"Commodore CDTV",
"159":"Nintendo DSi",
"161":"Windows Mixed Reality",
"162":"Oculus VR",
"163":"SteamVR",
"164":"Daydream",
"165":"PlayStation VR",
"166":"Pokémon mini",
"167":"PlayStation 5",
"169":"Xbox Series X|S",
"170":"Google Stadia",
"203":"DUPLICATE Stadia",
"236":"Exidy Sorcerer",
"237":"Sol-20",
"238":"DVD Player",
"239":"Blu-ray Player",
"240":"Zeebo",
"274":"PC-FX",
"306":"Satellaview",
"307":"Game \u0026 Watch",
"308":"Playdia",
"309":"Evercade",
"339":"Sega Pico",
"372":"OOParts",
"373":"Sinclair ZX81",
"374":"Sharp MZ-2200",
"375":"Epoch Cassette Vision",
"376":"Epoch Super Cassette Vision",
"377":"Plug \u0026 Play",
"378":"Gamate",
"379":"Game.com",
"380":"Casio Loopy",
"381":"Playdate",
"382":"Intellivision Amico",
"384":"Oculus Quest",
"385":"Oculus Rift",
"386":"Oculus Quest 2",
"387":"Oculus Go",
"388":"Gear VR",
"389":"AirConsole",
"390":"PlayStation VR2",
"405":"Windows Mobile"}
  byteArray = wrapper.api_request('games', 'fields platforms; where id = ' + str(gameID) + ';')
  myJson = byteArray.decode('utf8').replace("'", '"')
  myJson = myJson.replace('\n', ' ')
  if 'platforms' in myJson:
    platformList = re.search('"platforms": \[ (.+?) ]', myJson).group(1)
    platformList = platformList.replace(' ', '')
    platformList = platformList.split(',')
    found = ''
    platformList = [consoleID[id] for id in platformList]
    for _ in platformList:
      found = found + _ + ", "
    found = found.rstrip()
    found = found.strip(',')
    found = unicodeReplace(found)
    return found
  else:
    return "N/A"
def makeItem(jason):
  temp = ['','','','','','','','','','']
  game = jason.replace('\n', ' ')
  game = ' '.join(jason.split())
  if '"id"' in game:
    temp[0] = getID(game)
    time.sleep(.5)
  if '"name"' in game:
    temp[1] = getName(game)
    print("adding " + temp[1])
    time.sleep(.5)
  if '"collection"' in game:
    temp[2] = getCollection(game)
    time.sleep(.5)
  if '"first_release_date"' in game:
    temp[3] = getRelease(game)
    time.sleep(.5)
  if '"genres"' in game:
    temp[4] = getGenres(game)
    time.sleep(.5)
  if "themes" in game:
    temp[5] = getThemes(game)
    time.sleep(.5)
  if '"involved_companies"' in game:
    temp[6] = getCompanies(game)
    time.sleep(.5)
  if '"summary"' in game:
    temp[7]= getSummary(game)
    time.sleep(.5)
  if '"franchises"' in game:
    temp[8] = getFranchises(game)
    time.sleep(.5)
  if '"cover"' in game:
    temp[9] = getCover(temp[0])
    time.sleep(.5)
  return temp

Enter YOUR id and secret gotten from the Twitch developer page.

If you don't know what this means, or don't have these go here 

https://api-docs.igdb.com/#about 

and follow the create account sections

!!THIS SECTION MUST BE RUN EVERYTIME YOU CREATE A NEW LIST

In [3]:
#@title Authorization
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open('ID_INPUT').sheet1
IDlist = []
addedGames = []
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
for x in rows:
  IDlist.append(x[0])
  addedGames.append(x[1])
clientID = "p4drjdnuirwnzc9i1rs74fv8sdhtbi" #@param {type:"string"}
clientSecret = "anz0yqx9ppk7k649a1j80qclr9n1zv" #@param {type:"string"}
requestURL = "https://id.twitch.tv/oauth2/token?client_id=" + clientID + "&client_secret=" + clientSecret + "&grant_type=client_credentials"
getToken = requests.post(requestURL)
s = getToken.text
sI = s.split("\"")
accessToken = sI[3]
accessID = clientID
wrapper = IGDBWrapper(accessID, accessToken)

Search by title and select which games you want to add from a list using the number on the left (you can include multiple games each search just type in all the numbers you want to add at once, so to add 1, 2, and 5, type 125)

This section is optional, if you filled the A column of ID_INPUT with IDs yourself just skip this

In [ ]:
#@title Add ID directly
ID =  193322#@param {type:"number"}
clear_output()
print("Added games So Far:")
print(addedGames)
byteArray = wrapper.api_request('games', 'fields name; where id = ' + str(ID) + ';')
myJson = byteArray.decode('utf8').replace("'", '"')
if(myJson != '[]'):
  results = myJson.split('{')
  results.pop(0)
  resultsList = []
  for _ in results:
    tmp = ['', '']
    tmp[0] = getID(_)
    tmp[1] = getName(_)
    resultsList.append(tmp)

  for _ in resultsList:
    print("Added : " + str(_[1]) + '(' + str(_[0]) + ')')
    IDlist.append(_[0])
    addedGames.append(_[1])

  rangeCall = 'A1' + ':A' + str(len(IDlist))
  cell_list = worksheet.range(rangeCall)
  numin = 0
  for cell in cell_list:
    cell.value = IDlist[numin]
    numin += 1
  worksheet.update_cells(cell_list)

  rangeCall = 'B1' + ':B' + str(len(addedGames))
  cell_list = worksheet.range(rangeCall)
  numin = 0
  for cell in cell_list:
    cell.value = addedGames[numin]
    numin += 1
  worksheet.update_cells(cell_list)
else:
  print("No game found")


In [9]:
#@title Add IDs to ID_INPUT by searching
showCover = "Don't include Covers (Faster)" #@param ["Include Covers (Slower)", "Don't include Covers (Faster)"]
showPlatforms = True #@param {type:"boolean"}
cont = 1
while cont != 0:
  clear_output()
  print("Added games So Far:")
  print(addedGames)
  titleIN = input("(enter -1 to finish)\nTitle: ")
  if titleIN == '-1':
    cont = 0
    break
  byteArray = wrapper.api_request('games', 'fields name; limit 9; search "' + str(titleIN) + '";')
  myJson = byteArray.decode('utf8').replace("'", '"')
  results = myJson.split('{')
  results.pop(0)
  resultsList = []
  for _ in results:
    tmp = ['', '']
    tmp[0] = getID(_)
    tmp[1] = getName(_)
    resultsList.append(tmp)

  for _ in resultsList:
    if showPlatforms:
      print('\x1b[31m' + str(resultsList.index(_)+1) + '\x1b[0m' + ': ' + '\x1b[36m' + str(_[1]) + '(' + str(_[0]) + ') for ' + getPlatforms(_[0]) + '\x1b[0m')
    else:
      print('\x1b[31m' + str(resultsList.index(_)+1) + '\x1b[0m' + ': ' + '\x1b[36m' +str(_[1]) + '(' + str(_[0]) + ')'+ '\x1b[0m')
    if showCover == "Include Covers (Slower)":
      print(getCover(_[0]))
  addlist = (input("games to add:"))
  addlist = (set(addlist))
  cleaned = [ x for x in addlist if x.isdigit() ]
  addlist = cleaned

  if ('0' in addlist):
    continue
  for _ in addlist:
    IDlist.append(resultsList[int(_)-1][0])
    addedGames.append(resultsList[int(_)-1][1])

rangeCall = 'A1' + ':A' + str(len(IDlist))
cell_list = worksheet.range(rangeCall)
numin = 0
for cell in cell_list:
  cell.value = IDlist[numin]
  numin += 1
worksheet.update_cells(cell_list)

rangeCall = 'B1' + ':B' + str(len(addedGames))
cell_list = worksheet.range(rangeCall)
numin = 0
for cell in cell_list:
  cell.value = addedGames[numin]
  numin += 1
worksheet.update_cells(cell_list)

Added games So Far:
['Super Smash Bros. for Nintendo 3DS', 'Fire Emblem: Awakening', 'Monster Hunter Generations', 'Kid Icarus: Uprising', 'Yo-kai Watch', 'Kirby Triple Deluxe', 'Adventure Time: Finn and Jake Investigations', 'Crush3D', 'Scribblenauts Unlimited', 'Animal Crossing: New Leaf', 'Pokémon X', 'Pokémon Omega Ruby', 'Pokémon Sun', 'Paper Mario: Sticker Star', 'The Legend of Zelda: A Link Between Worlds', 'The Legend of Zelda: Ocarina of Time 3D', "The Legend of Zelda: Majora's Mask 3D", 'Hyrule Warriors: Legends', 'Professor Layton and the Miracle Mask', 'Professor Layton and the Azran Legacy', 'Professor Layton vs. Phoenix Wright: Ace Attorney', 'Code Name S.T.E.A.M.']
(enter -1 to finish)
Title: -1


{'spreadsheetId': '1EIuO_yd6lYaU8NeJnEugpHNmKPi3C_lOmzBy76JIiMU',
 'updatedCells': 22,
 'updatedColumns': 1,
 'updatedRange': 'Sheet1!B1:B22',
 'updatedRows': 22}

In [ ]:
#@title autosearching from list
showCover = "Don't include Covers (Faster)" #@param ["Include Covers (Slower)", "Don't include Covers (Faster)"]
showPlatforms = True #@param {type:"boolean"}
gameList = "" #@param {type:"string"}
#@markdown Make sure each game is seperated by a comma ( , )
gameList = gameList.split(',')
for titleIN in gameList:
  clear_output()
  print(titleIN)
  print("Added games So Far:")
  print(addedGames)
  byteArray = wrapper.api_request('games', 'fields name; limit 9; search "' + str(titleIN) + '";')
  myJson = byteArray.decode('utf8').replace("'", '"')
  results = myJson.split('{')
  results.pop(0)
  resultsList = []
  for _ in results:
    tmp = ['', '']
    tmp[0] = getID(_)
    tmp[1] = getName(_)
    resultsList.append(tmp)

  for _ in resultsList:
    if showPlatforms:
       print('\x1b[31m' + str(resultsList.index(_)+1) + '\x1b[0m' + ': ' + '\x1b[36m' + str(_[1]) + '(' + str(_[0]) + ') for ' + getPlatforms(_[0]) + '\x1b[0m')
    else:
      print('\x1b[31m' + str(resultsList.index(_)+1) + '\x1b[0m' + ': ' + '\x1b[36m' +str(_[1]) + '(' + str(_[0]) + ')'+ '\x1b[0m')
    if showCover == "Include Covers (Slower)":
      print(getCover(_[0]))
  addlist = (input("games to add:"))
  addlist = (set(addlist))
  cleaned = [ x for x in addlist if x.isdigit() ]
  addlist = cleaned

  if ('0' in addlist):
    continue
  for _ in addlist:
    IDlist.append(resultsList[int(_)-1][0])
    addedGames.append(resultsList[int(_)-1][1])

rangeCall = 'A1' + ':A' + str(len(IDlist))
cell_list = worksheet.range(rangeCall)
numin = 0
for cell in cell_list:
  cell.value = IDlist[numin]
  numin += 1
worksheet.update_cells(cell_list)

rangeCall = 'B1' + ':B' + str(len(addedGames))
cell_list = worksheet.range(rangeCall)
numin = 0
for cell in cell_list:
  cell.value = addedGames[numin]
  numin += 1
worksheet.update_cells(cell_list)

When ID_INPUT is filled with the IGDB IDs of the games you want (make sure they are all in column A) run this to fill in the rest of the game information, this takes a while so just let it run, and then go download ID_input, or copy the data to another file, just remember to clear ID_Input when you're done for the next time you want to use this program

In [93]:
#@title Populate ID_INPUT with game information
# Make the API call to populate list games from the IDs, then send them to function makeItem for formatting from JSON to csv
games = []

# refreshes getid and makes sure they are int
IDlist = []
rows = worksheet.get_all_values()
for x in rows:
  IDlist.append(int(x[0]))
cntr = 1
for ID in IDlist:
  rq = 'fields name, age_ratings, first_release_date, genres, involved_companies, summary, themes, url, collection, franchises, cover; where id = ' + str(ID) + ';'
  byteArray = wrapper.api_request(
              'games',
              rq
            )
  myJson = byteArray.decode('utf8').replace("'", '"')
  myJson = unicodeReplace(myJson)
  clear_output()
  progressBar(cntr, len(IDlist))
  cntr+=1
  games.append(makeItem(myJson))
  time.sleep(1)

#Stores the games in the original sheets file with headers
rangeCall = 'A1:I1'
cell_list = worksheet.range(rangeCall)
headerRow = ["ID", "Title", "Series", "Initial Release", "Genres", "Themes", "Companies", "Summary", "Franchise"]
numin = 0
for cell in cell_list:
  cell.value = headerRow[numin]
  numin +=1
worksheet.update_cells(cell_list)
clear_output()
print("Creating sheets file")
print(makeBar(21))
for _ in games:
  rangeCall = 'A' + str(games.index(_)+2) + ':J' + str(games.index(_)+2)
  cell_list = worksheet.range(rangeCall)
  numin = 0
  for cell in cell_list:
    cell.value = _[numin]
    numin += 1
  worksheet.update_cells(cell_list)
print("Finished")


Creating sheets file
|                     |
|                     |
Finished


From here you can open ID_INPUT and insert a row at the beggining, and use the formula =IMAGE(_cell_) with cell being the cover url cell add covers to your google sheet